In [4]:
%run lib/scrape_govuk.ipynb
metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')

import pyexcel
from io import BytesIO

ods_files = [f for f in metadata['files'] if f['type'] == 'ODS']

out = Path('out')
out.mkdir(exist_ok=True)

def load_sheet(ods_file_info, sheet_name):
    ods_file = BytesIO(session.get(ods_file_info['url']).content)
    return pyexcel.get_sheet(file_content=ods_file, file_type='ods', sheet_name=sheet_name)

for ods_file_info in ods_files:
    if ods_file_info['title'] == 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1':
        data = load_sheet(ods_file_info, 'vi_05')
        %run "Entry clearance visas granted by country of nationality(Vi_05).ipynb"
        entry_1 = tidy.copy()
        data = load_sheet(ods_file_info, 'vi_01_q')
        %run "Entry clearance visa applications and resolution by category ( vi_01_q).ipynb"
        entry_2 = tidy.copy()
        entry_1.rename(columns={'Year': 'Period'}, inplace=True)
        entry_1.replace(
            {'Period': r'$([0-9]{4})^'}, {'Period': r'year/\1'}, regex=True, inplace=True
        )
        for col in ['Type', 'Broad category', 'Applicant type', 'Category', 'Resolution']:
            entry_1[col] = 'All'
        # Todo: "World" or "Not UK"?
        entry_2['Country of nationality'] = 'World'
        pd.concat(entry_1, entry_2).to_csv(out / 'entry_visas.csv', index=False)
        
        

,Geographical region,Country of nationality,Year,Value


['Kosovo',
 'Asia South East',
 'St. Kitts and Nevis',
 'Anguilla (British)',
 'Africa North',
 'Niue',
 'Christmas Island',
 'EU Other',
 'EU 8',
 'Gibraltar (British)',
 'Montserrat (British)',
 'Pitcairn Islands (British)',
 'Asia Central',
 'America Central and South',
 'Cook Islands',
 'Guinea-Bissau',
 'Ireland',
 'Spain',
 'Turks and Caicos Islands (British)',
 'Curacao',
 'Korea (South)',
 'Wallis and Futuna',
 'Cyprus',
 'Surinam',
 'Former Yugoslavia',
 'Middle East',
 'Cambodia',
 'Virgin Islands (US)',
 'Europe Other',
 'Falkland Islands (British)',
 'Palau',
 'St. Pierre and Miquelon',
 'Burkina',
 'America North',
 'Cocos (Keeling) Islands',
 'Macau',
 'Korea (North)',
 'Sudan (South)',
 'American Samoa',
 'Central African Republic',
 'Samoa',
 'Africa Sub-Saharan',
 'British overseas citizens',
 'Micronesia',
 'Bonaire, Sint Eustatius and Saba',
 'Kiribati',
 'Cayman Islands (British)',
 'Faroe Islands',
 'Cyprus (Northern part of)',
 'Asia South',
 'St. Maarten (Dutch P

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"